In [1]:
import pandas as pd
from utils.load_db import LoadDb
from utils.load_municipios import LoadMunicipios
from utils.load_populacao import LoadPopulacao
from utils.load_bancos import LoadBancos
from utils.load_idh import LoadIDH

# Carregando os datasets para os dataframes

In [9]:
agencias = pd.read_excel('datasets/agencia.xlsx')
postos = pd.read_excel('datasets/postos.xlsx')
postose = pd.read_excel('datasets/postosEletronicos.xlsx')


# Carrengando os dataframes para um banco de dados sql

In [10]:
data = LoadDb('database.db')

## Salvando os dados de agencias no banco de dados

In [11]:
agencias

,NOME INSTITUIÇÃO,MUNICÍPIO,UF
0,CAIXA ECONOMICA FEDERAL ...,ABADIANIA ...,GO
1,ITAÚ UNIBANCO S.A. ...,ABADIANIA ...,GO
2,BANCO DO BRASIL S.A. ...,ABAETE ...,MG
3,CAIXA ECONOMICA FEDERAL ...,ABAETE ...,MG
4,ITAÚ UNIBANCO S.A. ...,ABAETE ...,MG
...,...,...,...
19314,BANCO BRADESCO S.A. ...,XIQUE-XIQUE ...,BA
19315,BANCO DO BRASIL S.A. ...,ZE DOCA ...,MA
19316,CAIXA ECONOMICA FEDERAL ...,ZE DOCA ...,MA
19317,BANCO DO NORDESTE DO BRASIL S.A. ...,ZE DOCA ...,MA


In [ ]:
for i in agencias.values:
    data.insert_agencia(i[0], i[1], i[2])

## Salvando os dados de postos no banco de dados

In [ ]:
for i in postos.values:
    data.insert_postos(i[0], i[1], i[2])

## Salvando os dados de postos eletronicos no banco de dados

In [ ]:
for i in postose.values:
    data.insert_postoseletronicos(i[0], i[1], i[2])

In [ ]:
data.close()

# Carregando os municipios em outra tabela

In [14]:
data = LoadMunicipios('database.db')

In [15]:
agencias = pd.read_sql('SELECT DISTINCT municipio, uf FROM agencias', con=data.conn)
postos = pd.read_sql('SELECT DISTINCT municipio, uf FROM postos', con=data.conn)
postose = pd.read_sql('SELECT DISTINCT municipio, uf FROM postos_eletronicos', con=data.conn)

In [ ]:
for i in agencias.values:
    result = data.select(i[0])
    
    data.insert_municipio(i[0], i[1])

In [ ]:
for i in postos.values:
    result = data.select(i[0])

    data.insert_municipio(i[0], i[1], result)

In [ ]:
for i in postose.values:
    result = data.select(i[0])

    data.insert_municipio(i[0], i[1], result)

In [16]:
data.close()

# Carregando os dados de quantidade de agencias, postos e postos eletronicos por instituição

In [ ]:
data = LoadBancos('database.db')

In [ ]:
agencias = pd.read_sql('SELECT nome_instituicao, municipio,  count(id_agencia) FROM agencias GROUP BY nome_instituicao, municipio ORDER BY municipio', con=data.conn)

for i in agencias.values:
    result = data.select(i[1])

    data.insert_bancos_agencias(i[0], i[2], result)

In [ ]:
postos = pd.read_sql('SELECT nome_instituicao, municipio, count(id_postos) FROM postos GROUP BY nome_instituicao, municipio ORDER BY municipio', con=data.conn)

for i in postos.values:
    result = data.select(i[1])

    data.insert_or_update_postos_e(i[0], i[2], result)

In [ ]:
postose = pd.read_sql('SELECT nome_instituicao, municipio, count(id_postose) FROM postos_eletronicos GROUP BY nome_instituicao, municipio ORDER BY municipio', con=data.conn)

for i in postose.values:
    result = data.select(i[1])

    data.insert_or_update_postos_e(i[0], i[2], result)

In [ ]:
data.close()

# Carregando dados de população

In [17]:
data = LoadPopulacao('database.db')

In [18]:
populacao = pd.read_excel('datasets/populacao.xls', sheet_name='municipios')
populacao

,UF,MUNICÍPIO,POPULAÇÃO ESTIMADA,Coeficiente de ajuste populacional
0,GO,ABADIA DE GOIAS,8958,25
1,MG,ABADIA DOS DOURADOS,7006,10
2,GO,ABADIANIA,20461,10
3,MG,ABAETE,23250,10
4,PA,ABAETETUBA,159080,15
...,...,...,...,...
5565,BA,XIQUE-XIQUE,46523,10
5566,PB,ZABELE,2255,5
5567,SP,ZACARIAS,2752,5
5568,MA,ZE DOCA,51956,10


In [23]:
for i in populacao.values:
    result = data.select(i[1])

    data.insert(i[2], i[3], result)

AttributeError: 'LoadPopulacao' object has no attribute 'select'

In [2]:
data.close()

NameError: name 'data' is not defined

# Carregando dados de idh

In [3]:
data = LoadIDH('database.db')

In [4]:
idh = pd.read_excel('datasets/idh.xlsx')
idh


,MUNICÍPIO,IDHM
0,ABADIA DE GOIAS,0.708
1,ABADIA DOS DOURADOS,0.689
2,ABADIANIA,0.689
3,ABAETE,0.698
4,ABAETETUBA,0.628
...,...,...
5560,XIQUE-XIQUE,0.585
5561,ZABELE,0.623
5562,ZACARIAS,0.729
5563,ZE DOCA,0.595


In [5]:
for i in idh.values:
    result = data.select(i[0])

    data.insert(i[1], result)

In [11]:
data.close()